In [2]:
import easyocr
import cv2 as cv

reader = easyocr.Reader(['ko', 'en', 'ch_sim', 'ja'])
result = reader.readtext(r'C:\ex\prj_poster\title_detect\combined_edges.jpg')
result

Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


ValueError: Chinese_sim is only compatible with English, try lang_list=["ch_sim","en"]

In [5]:
import easyocr

# 올바른 언어 코드 사용
reader = easyocr.Reader(['ko', 'en'])  # 'zh' 사용

# 텍스트 추출할 이미지 파일 경로
image_path = r'C:\\ex\\prj_poster\\title_detect\\combined_edges.jpg'

# 이미지에서 텍스트 추출
results = reader.readtext(image_path)

# 결과 출력
for (bbox, text, prob) in results:
    print(f"Detected text: {text}, Confidence: {prob}")

Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


Detected text: 모조관, Confidence: 0.026382442851082842
Detected text: [ 제내헤움[다[, Confidence: 0.05200601536962351
Detected text: 떼기로록, Confidence: 0.03408586233854294
Detected text: 노끗, Confidence: 0.0022432237705254424
Detected text: '정어액 슬쩍 , Confidence: 0.0022394028965984697
Detected text: "Plupafiiil, Confidence: 0.0018044526903874417
Detected text: 도중;, Confidence: 0.1965672533314896
Detected text: K매떨, Confidence: 0.013250079912947623
Detected text: nn동 I, Confidence: 0.008182931881100967


In [ ]:
import cv2
import numpy as np
import easyocr
import matplotlib.pyplot as plt

# 이미지 파일 경로
image_path = 'img.jpg'

# 이미지 로드
image = cv2.imread(image_path)

# 1. 가우시안 블러
blurred = cv2.GaussianBlur(image, (5, 5), 0)  # 커널 크기를 (5, 5)로 설정

# 2. 소벨 엣지 검출
sobel_x = cv2.Sobel(blurred, cv2.CV_64F, 1, 0, ksize=3)  # x 방향 소벨 필터, 커널 크기 3
sobel_y = cv2.Sobel(blurred, cv2.CV_64F, 0, 1, ksize=3)  # y 방향 소벨 필터
sobel_edges = cv2.magnitude(sobel_x, sobel_y)  # 소벨 엣지 결합
sobel_edges = np.uint8(sobel_edges)  # 정수형으로 변환

# 3. 캐니 엣지 검출
threshold1 = 50  # 첫 번째 임계값
threshold2 = 150  # 두 번째 임계값
canny_edges = cv2.Canny(sobel_edges, threshold1, threshold2)  # 캐니 엣지 검출

# 4. 이미지 시각화
plt.figure(figsize=(12, 8))
plt.subplot(1, 3, 1)
plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
plt.title('Original Image')
plt.axis('off')

plt.subplot(1, 3, 2)
plt.imshow(sobel_edges, cmap='gray')
plt.title('Sobel Edges')
plt.axis('off')

plt.subplot(1, 3, 3)
plt.imshow(canny_edges, cmap='gray')
plt.title('Canny Edges')
plt.axis('off')

plt.show()

# EAST 모델을 통한 텍스트 감지
east_model_path = "frozen_east_text_detection.pb"

# 입력 크기 설정
input_width = 320
input_height = 320
conf_threshold = 0.5
nms_threshold = 0.4

# 블롭으로 변환하기 전에 컬러 형식 변환
canny_edges_color = cv2.cvtColor(canny_edges, cv2.COLOR_GRAY2BGR)  # 그레이스케일을 RGB로 변환

# 블롭으로 변환
blob = cv2.dnn.blobFromImage(canny_edges_color, 1.0, (input_width, input_height),
                             (123.68, 116.78, 103.94), swapRB=True, crop=False)

# EAST 모델 로드
net = cv2.dnn.readNet(east_model_path)

# 입력을 네트워크에 설정
net.setInput(blob)

# 네트워크의 출력 레이어 정의
layer_names = [
    "feature_fusion/Conv_7/Sigmoid",  # 텍스트 확률 지도
    "feature_fusion/concat_3"         # 텍스트 경계 박스 좌표
]

# 모델 추론
(scores, geometry) = net.forward(layer_names)

# 텍스트 상자 계산 함수
def decode_predictions(scores, geometry, conf_threshold):
    (num_rows, num_cols) = scores.shape[2:4]
    boxes = []
    confidences = []

    for y in range(num_rows):
        scores_data = scores[0, 0, y]
        x0_data = geometry[0, 0, y]
        x1_data = geometry[0, 1, y]
        x2_data = geometry[0, 2, y]
        x3_data = geometry[0, 3, y]
        angles_data = geometry[0, 4, y]

        for x in range(num_cols):
            if scores_data[x] < conf_threshold:
                continue

            angle = angles_data[x]
            cos = np.cos(angle)
            sin = np.sin(angle)

            h = x0_data[x] + x2_data[x]
            w = x1_data[x] + x3_data[x]

            end_x = int(x * 4.0 + (cos * x1_data[x]) + (sin * x2_data[x]))
            end_y = int(y * 4.0 - (sin * x1_data[x]) + (cos * x2_data[x]))
            start_x = int(end_x - w)
            start_y = int(end_y - h)

            # 상자 형식을 (start_x, start_y, width, height)로 변환
            boxes.append([start_x, start_y, int(w), int(h)])
            confidences.append(float(scores_data[x]))

    return (boxes, confidences)

# 감지된 텍스트 상자 및 신뢰도 값 계산
(boxes, confidences) = decode_predictions(scores, geometry, conf_threshold)

# NMS (Non-Maximum Suppression)를 통해 겹치는 박스 제거
indices = cv2.dnn.NMSBoxes(boxes, confidences, conf_threshold, nms_threshold)

# 텍스트 상자 그리기
orig = image.copy()  # 원본 이미지를 복사
if len(indices) > 0:
    for i in indices.flatten():
        (start_x, start_y, w, h) = boxes[i]
        
        # 텍스트 영역 그리기
        end_x = start_x + w
        end_y = start_y + h
        cv2.rectangle(orig, (start_x, start_y), (end_x, end_y), (0, 255, 0), 2)

# 결과 출력
plt.figure(figsize=(10, 10))
plt.imshow(cv2.cvtColor(orig, cv2.COLOR_BGR2RGB))
plt.title("Detected Text Areas")
plt.axis("off")
plt.show()


### 1. **EAST (Efficient and Accurate Scene Text Detector)**

- **설명**: EAST 모델은 이미지 내 텍스트의 위치를 ​​감지하는 데 매우 효과적입니다. 텍스트 경계 상자를 예측하여 텍스트가 있는 영역을 찾습니다.
- **장점**: 다양한 텍스트 방향과 크기를 처리할 수 있으며, 빠른 속도로 실시간 처리에 적합합니다.
- **사용법**: OpenCV와 함께 사용할 수 있으며, 텍스트 영역을 사각형으로 그려 시각화할 수 있습니다.

### 2. **Tesseract OCR**

- **설명**: Tesseract는 Google이 지원하는 오픈 소스 OCR 엔진으로, 텍스트 인식에 매우 강력합니다.
- **장점**: 다양한 언어를 지원하며, 전처리 과정을 통해 텍스트 인식 정확도를 높일 수 있습니다.
- **사용법**: 이미지에서 텍스트를 추출하는 데 간단한 API를 제공합니다.

### 3. **EasyOCR**

- **설명**: EasyOCR은 PyTorch를 기반으로 한 OCR 라이브러리로, 많은 언어를 지원합니다.
- **장점**: 사용자 친화적인 인터페이스와 다양한 언어 지원(영어, 한국어, 중국어, 일본어 등)을 제공합니다.
- **사용법**: 설치가 간단하고, 다양한 예제를 통해 쉽게 사용할 수 있습니다.

### 4. **PaddleOCR**

- **설명**: PaddleOCR은 PaddlePaddle 기반의 OCR 라이브러리로, 실시간 OCR을 지원합니다.
- **장점**: 높은 정확도와 빠른 속도를 제공하며, 여러 언어를 지원합니다.
- **사용법**: 다양한 모델을 통해 복잡한 이미지에서도 텍스트를 추출할 수 있습니다.

### 5. **Deep Text Recognition Model**

- **설명**: 이 모델은 텍스트를 인식하는 데 딥러닝을 활용합니다. 이미지의 복잡성을 처리하기 위해 CNN과 RNN을 조합합니다.
- **장점**: 복잡한 텍스트 배열에서도 잘 작동합니다.
- **사용법**: 학습된 모델을 통해 직접 구현할 수 있으며, 특정 데이터셋에 맞춰 재학습할 수 있습니다.

### 6. **Google Vision API**

- **설명**: Google Cloud에서 제공하는 API로, 텍스트 감지 및 OCR 기능을 지원합니다.
- **장점**: 다양한 텍스트 유형을 인식할 수 있으며, 클라우드 기반으로 작동하여 신뢰성이 높습니다.
- **사용법**: API를 통해 손쉽게 텍스트를 추출할 수 있으며, 유료 서비스입니다.